In [1]:
import warnings
import sys
import os

warnings.filterwarnings('ignore')
os.environ["CUDA_VISIBLE_DEVICES"] = ''
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '.95'
sys.path.append('/mnt/local/data/vtrifonov/prec-learning-Notay-loss/')

In [2]:
import jax.numpy as jnp
from jax import random, vmap
from jax.experimental import sparse as jsparse
from jax.scipy.sparse.linalg import cg as jcg

import numpy as np
from scipy.sparse.linalg import LinearOperator, cg
from scipy.sparse import tril, triu, eye

import matplotlib.pyplot as plt
from functools import partial
from time import perf_counter
import ilupp

from linsolve.precond import llt_prec_trig_solve, llt_inv_prec
from utils import factorsILUp, batchedILUp, ILU2, jILU2, iter_per_residual, jBCOO_to_scipyCSR
from linsolve.cg import ConjGrad
from data.qtt import div_k_grad, scipy_validation, solve_precChol, solve_precLU

In [3]:
A, b, x, k_stats = div_k_grad(1, 32, 'gaussian', 1.)
print(k_stats)

A_ = jBCOO_to_scipyCSR(A[0, ...])
b_ = np.asarray(b[0, ...])

CUDA backend failed to initialize: jaxlib/cuda/versions_helpers.cc:98: operation cuInit(0) failed: CUDA_ERROR_NO_DEVICE (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


{'min': 342.0, 'max': 342.0, 'mean': 342.0}


In [4]:
d_tol = scipy_validation(A_, b_, prec_f=lambda x: x)
print(f'CG: {d_tol}')

CG: {'1e-3': 269, '1e-6': 344, '1e-12': 469}


# ILU with recursive call of ILU(0)

In [5]:
L, U = factorsILUp(A_, p=0)
prec_f = partial(solve_precLU, L=L, U=U)
d_tol = scipy_validation(A_, b_, prec_f)
print(f'PCG: {d_tol}')
print(f'NNZ(L) = {L.nnz*100 / L.shape[-1] ** 2:.3f}%')

PCG: {'1e-3': 45, '1e-6': 57, '1e-12': 74}
NNZ(L) = 0.287%


In [6]:
L, U = factorsILUp(A_, p=1)
prec_f = partial(solve_precLU, L=L, U=U)
d_tol = scipy_validation(A_, b_, prec_f)
print(f'PCG: {d_tol}')
print(f'NNZ(L) = {L.nnz*100 / L.shape[-1] ** 2:.3f}%')

PCG: {'1e-3': 45, '1e-6': 57, '1e-12': 74}
NNZ(L) = 0.379%


In [7]:
L, U = factorsILUp(A_, p=2)
prec_f = partial(solve_precLU, L=L, U=U)
d_tol = scipy_validation(A_, b_, prec_f)
print(f'PCG: {d_tol}')
print(f'NNZ(L) = {L.nnz*100 / L.shape[-1] ** 2:.3f}%')

PCG: {'1e-3': 45, '1e-6': 57, '1e-12': 74}
NNZ(L) = 0.386%


# ILU(2) from Fortran

In [8]:
# s = perf_counter()
# sILU = ILU2(A_)
# print(perf_counter() - s)

In [9]:
# LU = A_.copy()
# LU.data = sILU
# L = tril(LU, k=-1) + eye(LU.shape[0])
# U = triu(LU)

In [10]:
# Do not converge

# prec_f = partial(solve_precLU, L=L, U=U)
# d_tol = scipy_validation(A_, b_, prec_f)
# print(f'CG with from matlab code ILU(2): {d_tol}')

# IC(0)

In [11]:
L = ilupp.ichol0(A_)

prec_f = partial(solve_precChol, L=L, U=L)
d_tol = scipy_validation(A_, b_, prec_f)
print(f'PCG: {d_tol}')
print(f'NNZ(L) = {L.nnz*100 / L.shape[-1] ** 2:.3f}%')

PCG: {'1e-3': 45, '1e-6': 57, '1e-12': 74}
NNZ(L) = 0.287%


# ICt

In [12]:
L = ilupp.icholt(A_, add_fill_in=1, threshold=1e-4)

prec_f = partial(solve_precChol, L=L, U=L)
d_tol = scipy_validation(A_, b_, prec_f)
print(f'PCG: {d_tol}')
print(f'NNZ(L) = {L.nnz*100 / L.shape[-1] ** 2:.3f}%')

PCG: {'1e-3': 27, '1e-6': 34, '1e-12': 44}
NNZ(L) = 0.378%


In [13]:
L = ilupp.icholt(A_, add_fill_in=2, threshold=1e-4)

prec_f = partial(solve_precChol, L=L, U=L)
d_tol = scipy_validation(A_, b_, prec_f)
print(f'PCG: {d_tol}')
print(f'NNZ(L) = {L.nnz*100 / L.shape[-1] ** 2:.3f}%')

PCG: {'1e-3': 20, '1e-6': 25, '1e-12': 33}
NNZ(L) = 0.475%


In [14]:
L = ilupp.icholt(A_, add_fill_in=5, threshold=1e-4)

prec_f = partial(solve_precChol, L=L, U=L)
d_tol = scipy_validation(A_, b_, prec_f)
print(f'PCG: {d_tol}')
print(f'NNZ(L) = {L.nnz*100 / L.shape[-1] ** 2:.3f}%')

PCG: {'1e-3': 12, '1e-6': 15, '1e-12': 22}
NNZ(L) = 0.755%


In [15]:
L = ilupp.icholt(A_, add_fill_in=10, threshold=1e-4)

prec_f = partial(solve_precChol, L=L, U=L)
d_tol = scipy_validation(A_, b_, prec_f)
print(f'PCG: {d_tol}')
print(f'NNZ(L) = {L.nnz*100 / L.shape[-1] ** 2:.3f}%')

PCG: {'1e-3': 8, '1e-6': 10, '1e-12': 15}
NNZ(L) = 1.178%


In [16]:
L = ilupp.icholt(A_, add_fill_in=15, threshold=1e-4)

prec_f = partial(solve_precChol, L=L, U=L)
d_tol = scipy_validation(A_, b_, prec_f)
print(f'PCG: {d_tol}')
print(f'NNZ(L) = {L.nnz*100 / L.shape[-1] ** 2:.3f}%')

PCG: {'1e-3': 5, '1e-6': 7, '1e-12': 11}
NNZ(L) = 1.529%


In [17]:
L = ilupp.icholt(A_, add_fill_in=20, threshold=1e-4)

prec_f = partial(solve_precChol, L=L, U=L)
d_tol = scipy_validation(A_, b_, prec_f)
print(f'PCG: {d_tol}')
print(f'NNZ(L) = {L.nnz*100 / L.shape[-1] ** 2:.3f}%')

PCG: {'1e-3': 5, '1e-6': 6, '1e-12': 10}
NNZ(L) = 1.776%


In [18]:
L = ilupp.icholt(A_, add_fill_in=30, threshold=1e-4)

prec_f = partial(solve_precChol, L=L, U=L)
d_tol = scipy_validation(A_, b_, prec_f)
print(f'PCG: {d_tol}')
print(f'NNZ(L) = {L.nnz*100 / L.shape[-1] ** 2:.3f}%')

PCG: {'1e-3': 4, '1e-6': 6, '1e-12': 9}
NNZ(L) = 1.908%


# ILUt

In [19]:
L, U = ilupp.ilut(A_, fill_in=1, threshold=1e-4)

prec_f = partial(solve_precLU, L=L, U=U)
d_tol = scipy_validation(A_, b_, prec_f)
print(f'PCG: {d_tol}')
print(f'NNZ(L) = {L.nnz*100 / L.shape[-1] ** 2:.3f}%')

PCG: {'1e-3': 143, '1e-6': 179, '1e-12': 230}
NNZ(L) = 0.098%


In [20]:
L, U = ilupp.ilut(A_, fill_in=2, threshold=1e-4)

prec_f = partial(solve_precLU, L=L, U=U)
d_tol = scipy_validation(A_, b_, prec_f)
print(f'PCG: {d_tol}')
print(f'NNZ(L) = {L.nnz*100 / L.shape[-1] ** 2:.3f}%')

PCG: {'1e-3': 10240, '1e-6': 10240, '1e-12': 10240}
NNZ(L) = 0.195%


In [21]:
L, U = ilupp.ilut(A_, fill_in=3, threshold=1e-4)

prec_f = partial(solve_precLU, L=L, U=U)
d_tol = scipy_validation(A_, b_, prec_f)
print(f'PCG: {d_tol}')
print(f'NNZ(L) = {L.nnz*100 / L.shape[-1] ** 2:.3f}%')

PCG: {'1e-3': 10240, '1e-6': 10240, '1e-12': 10240}
NNZ(L) = 0.290%


In [22]:
L, U = ilupp.ilut(A_, fill_in=4, threshold=1e-4)

prec_f = partial(solve_precLU, L=L, U=U)
d_tol = scipy_validation(A_, b_, prec_f)
print(f'PCG: {d_tol}')
print(f'NNZ(L) = {L.nnz*100 / L.shape[-1] ** 2:.3f}%')

PCG: {'1e-3': 10240, '1e-6': 10240, '1e-12': 10240}
NNZ(L) = 0.384%


In [23]:
L, U = ilupp.ilut(A_, fill_in=5, threshold=1e-4)

prec_f = partial(solve_precLU, L=L, U=U)
d_tol = scipy_validation(A_, b_, prec_f)
print(f'PCG: {d_tol}')
print(f'NNZ(L) = {L.nnz*100 / L.shape[-1] ** 2:.3f}%')

PCG: {'1e-3': 24, '1e-6': 37, '1e-12': 73}
NNZ(L) = 0.479%


In [24]:
L, U = ilupp.ilut(A_, fill_in=6, threshold=1e-4)

prec_f = partial(solve_precLU, L=L, U=U)
d_tol = scipy_validation(A_, b_, prec_f)
print(f'PCG: {d_tol}')
print(f'NNZ(L) = {L.nnz*100 / L.shape[-1] ** 2:.3f}%')

PCG: {'1e-3': 19, '1e-6': 32, '1e-12': 86}
NNZ(L) = 0.574%


In [25]:
L, U = ilupp.ilut(A_, fill_in=7, threshold=1e-4)

prec_f = partial(solve_precLU, L=L, U=U)
d_tol = scipy_validation(A_, b_, prec_f)
print(f'PCG: {d_tol}')
print(f'NNZ(L) = {L.nnz*100 / L.shape[-1] ** 2:.3f}%')

PCG: {'1e-3': 16, '1e-6': 26, '1e-12': 87}
NNZ(L) = 0.668%


In [26]:
L, U = ilupp.ilut(A_, fill_in=10, threshold=1e-4)

prec_f = partial(solve_precLU, L=L, U=U)
d_tol = scipy_validation(A_, b_, prec_f)
print(f'PCG: {d_tol}')
print(f'NNZ(L) = {L.nnz*100 / L.shape[-1] ** 2:.3f}%')

PCG: {'1e-3': 11, '1e-6': 17, '1e-12': 33}
NNZ(L) = 0.950%


In [27]:
L, U = ilupp.ilut(A_, fill_in=15, threshold=1e-4)

prec_f = partial(solve_precLU, L=L, U=U)
d_tol = scipy_validation(A_, b_, prec_f)
print(f'PCG: {d_tol}')
print(f'NNZ(L) = {L.nnz*100 / L.shape[-1] ** 2:.3f}%')

PCG: {'1e-3': 7, '1e-6': 10, '1e-12': 18}
NNZ(L) = 1.392%


In [28]:
L, U = ilupp.ilut(A_, fill_in=20, threshold=1e-4)

prec_f = partial(solve_precLU, L=L, U=U)
d_tol = scipy_validation(A_, b_, prec_f)
print(f'PCG: {d_tol}')
print(f'NNZ(L) = {L.nnz*100 / L.shape[-1] ** 2:.3f}%')

PCG: {'1e-3': 5, '1e-6': 7, '1e-12': 12}
NNZ(L) = 1.760%


In [29]:
L, U = ilupp.ilut(A_, fill_in=30, threshold=1e-4)

prec_f = partial(solve_precLU, L=L, U=U)
d_tol = scipy_validation(A_, b_, prec_f)
print(f'PCG: {d_tol}')
print(f'NNZ(L) = {L.nnz*100 / L.shape[-1] ** 2:.3f}%')

PCG: {'1e-3': 3, '1e-6': 5, '1e-12': 8}
NNZ(L) = 2.165%
